# RAG-Chatbot 

In [42]:
import pandas as pd
import os
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from neo4j import Query, GraphDatabase, RoutingControl, Result
from dotenv import load_dotenv
import gradio as gr
import time
from IPython.display import display, HTML
import warnings
from json import loads, dumps
warnings.filterwarnings('ignore')

## Get Credentials

In [6]:
env_file = 'credentials.env'

In [7]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
    EMBEDDINGS_MODEL = os.getenv('EMBEDDINGS_MODEL')
else:
    print(f"File {env_file} not found.")

## Setup Connection to Database

Setup connection to the database with the Python Driver

In [8]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

In [9]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,Count
0,902


## Create RAG-application

For the the chatbot we both need an Embedding-model and LLM. Create both below:

In [10]:
embedding_model = OpenAIEmbeddings(
    model=EMBEDDINGS_MODEL,
    openai_api_key=OPENAI_API_KEY
)

In [11]:
embedding_model.model

'text-embedding-ada-002'

In [12]:
llm = ChatOpenAI(temperature=0, model=LLM)
llm.model_name

'gpt-4o'

### Retrieval Queries

To illustrate the difference between a "Regular" Vector Search and GraphRAG we create different retrieval queries.

In [85]:
def get_context_vector_search(search_prompt):
    query_vector = embedding_model.embed_query(search_prompt)
    
    similarity_query = """ 
        CALL db.index.vector.queryNodes("chunk-embeddings", 5, $query_vector) YIELD node, score
        WITH node as chunk, score ORDER BY score DESC
        MATCH (d:Document)<-[:PART_OF]-(chunk)
        RETURN score, d.file_name as file_name, chunk.id as chunk_id, chunk.page as page, chunk.chunk AS chunk
       """
    results = driver.execute_query(
        similarity_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        query_vector=query_vector,
        result_transformer_= lambda r: r.to_df()
    )
    
    results = results.to_json(orient="records")
    parsed = loads(results)
    context = dumps(parsed, indent=4)

    return context

In [80]:
def get_context_graphrag(search_prompt):

    query_vector = embedding_model.embed_query(search_prompt)
    
    similarity_query = """ 
        CALL db.index.vector.queryNodes("chunk-embeddings", 5, $query_vector) YIELD node, score
        WITH node as chunk, score ORDER BY score DESC
        MATCH (d:Document)<-[:PART_OF]-(chunk)
        RETURN score, d.file_name as file_name, chunk.id as chunk_id, chunk.page as page, chunk.chunk AS chunk
       """
    results = driver.execute_query(
        similarity_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        query_vector=query_vector,
        result_transformer_= lambda r: r.to_df()
    )

    chunk_ids = list(set(results['chunk_id'].to_list()))

    results = results.to_json(orient="records")
    parsed = loads(results)
    context = dumps(parsed, indent=4)

    definition_query = """    
        MATCH (c:Chunk)-[:MENTIONS]->(d:Definition)
        WHERE c.id in $chunk_ids
        RETURN DISTINCT d.term as term, d.description as description
    """
    results = driver.execute_query(
        definition_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        chunk_ids=chunk_ids,
        result_transformer_= lambda r: r.to_df()
    )
    results = results.to_json(orient="records")
    parsed = loads(results)
    definitions = dumps(parsed, indent=4)
    return context, definitions

Function to retrieve the client name from a client id

### Prompts 

Prompt for vector search (without definitions)

In [81]:
def generate_prompt_vector_search(search_prompt, context):
    prompt_template = """

    You are a chatbot on Rabobank product. Your goal is to help people with questions on product policies.  
    A user will come to you with questions on their policy. Their questions must be answered based on the relevant documents of the policy.
    
    The question is the following: 
    {search_prompt}
    Always respond in the language in which the question was asked. So, do not respond in a different language.
    
    The context is the following: 
    {context}

    Please end your message with listing your sources with file name and page number. 
    """
    prompt = PromptTemplate.from_template(prompt_template)
    
    theprompt = prompt.format_prompt(search_prompt=search_prompt, context=context)
    return theprompt

In [82]:
def generate_prompt_graphrag(search_prompt, context, definitions):
    prompt_template = """

    You are a chatbot on Rabobank product. Your goal is to help people with questions on product policies.  
    A user will come to you with questions on their policy. Their questions must be answered based on the relevant documents of the policy.
    
    The question is the following: 
    {search_prompt}
    Always respond in the language in which the question was asked. So, do not respond in a different language.
    
    The context is the following: 
    {context}

    The definitions are the following: 
    {definitions}
    
    Please end your message with listing your sources with file name and page number. 
    """
    prompt = PromptTemplate.from_template(prompt_template)
    
    theprompt = prompt.format_prompt(search_prompt=search_prompt, context=context, definitions=definitions)
    return theprompt

Prompt for GraphRAG

## Some examples to test the models

For every example there can be chosen between GraphRAG and vector search. 

In [86]:
search_prompt = 'Wat wordt bedoelt met de Rabofoon?'

context = get_context_vector_search(search_prompt)
theprompt = generate_prompt_vector_search(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

================================== Ai Message ==================================

De Rabofoon is een dienst van Rabobank waarmee u betaalopdrachten kunt geven via een vaste of mobiele telefoon. U kunt de Rabofoon gebruiken als deze voor u is geactiveerd. Met de toetsen van de telefoon geeft u de betaalopdracht en vervolgens geeft u toestemming volgens de aanwijzingen van de Rabofoon. Zodra de bevestigingstoets is ingetoetst, is de betaalopdracht ontvangen en kan deze niet meer worden ingetrokken. Voor overboekingen via de Rabofoon moet het IBAN als unieke identificator worden gebruikt. Daarnaast is het gebruik van de Rabofoon gratis, maar er kunnen wel kosten in rekening worden gebracht voor de telefoonverbinding.

Bronnen:
- Payment and Online Services Terms Sept 2022.pdf, pagina 51
- Payment and Online Services Terms Sept 2022.pdf, pagina 30
- Rabo SpaarRekening 2020.pdf, pagina 3


In [87]:
search_prompt = 'Wat wordt bedoelt met de Rabofoon?'

context, definitions = get_context_graphrag(search_prompt)
theprompt = generate_prompt_graphrag(search_prompt, context, definitions)
llm(theprompt.to_messages()).pretty_print()

================================== Ai Message ==================================

De Rabofoon is een dienst van Rabobank waarmee u betaalopdrachten kunt geven via de telefoon. U gebruikt de toetsen van de telefoon om de opdracht in te voeren en volgt de aanwijzingen van de Rabofoon om de opdracht te bevestigen. Zodra de systemen van Rabobank vastleggen dat u de bevestigingstoets heeft ingetoetst, wordt de betaalopdracht ontvangen en kan deze niet meer worden ingetrokken. Voor overboekingen met de Rabofoon moet het IBAN als unieke identificator worden gebruikt. 

Bronnen:
- "Payment and Online Services Terms Sept 2022.pdf", pagina 51
- "Payment and Online Services Terms Sept 2022.pdf", pagina 30


In [52]:
search_prompt = 'Wat is fuseren?'

context = get_context_vector_search(search_prompt)
theprompt = generate_prompt_vector_search(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

[
    {
        "score": 0.8935089111,
        "file_name": "Rabo SpaarRekening 2020.pdf",
        "chunk_id": 37,
        "page": 12,
        "chunk": "Pagina 13/14\nAlgemene voorwaarden Rabo SpaarRekening 2020  November 2023\n24. Fusie, splitsing, contractsoverneming of overdracht\n 1 Fusie en splitsing\n   Wij kunnen samengaan met een andere rechtspersoon. Dat heet fuseren. Wij kunnen ook in onderdelen \nworden opgesplitst. Dat heet splitsen. Fuseren of splitsen wij? Dan kunnen onze rechtsopvolgers zelfstandig \nen ieder voor het geheel:\n \u2022 alle rechten en bevoegdheden tegenover u uitoefenen\n \u2022 al onze verplichtingen tegenover u nakomen.\n 2 Contractsoverneming\n   Wij mogen de rechtsverhouding met u en de rechten, plichten en nevenrechten die daarbij horen overdragen \naan een ander. Dat kan helemaal of voor een deel. Dat heet contractsoverneming. U geeft ons daarvoor nu al \ntoestemming door het sluiten van de overeenkomst.\n 3 Overdracht\n   Wij mogen de vorderingen o

In [53]:
search_prompt = 'Mag ik rood staan op mijn rekening?'

context = get_context_vector_search(search_prompt)
theprompt = generate_prompt_vector_search(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

[
    {
        "score": 0.9281921387,
        "file_name": "Payment and Online Services Terms Sept 2022.pdf",
        "chunk_id": 116,
        "page": 19,
        "chunk": "dit niet tot korting op het tarief. \n \n4. Wilt u niet langer online bankieren? Dan kan dat tot gevolg hebben dat uw \nbetaalpakket niet meer geschikt is en u een ander pakket moet kiezen. Maakt u \ngeen keuze? Dan mogen wij uw betaalpakket omzetten.\n49. Zonder krediet rood staan op de rekening\n1. Als u geen krediet heeft, kunt u toch rood komen te staan op de rekening. \nBijvoorbeeld doordat wij de kosten van het betaalpakket afboeken van de \nrekening. Wij noemen dat ongeoorloofd rood staan. U betaalt hierover een \nvariabele rente.  \nWij kunnen deze rente altijd wijzigen.\n2. Deze variabele rente is opgebouwd uit de volgende componenten: een \nbasistarief, opslagen in verband met ontwikkelingen op de kapitaalmarkten \nen kapitaalkosten, eventueel individuele risico-opslagen, doorlopende kosten \n(bijvoorbeel

In [54]:
search_prompt = 'Mag ik rood staan op mijn rekening?'

context = get_context_vector_search(search_prompt)
theprompt = generate_prompt_vector_search(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

[
    {
        "score": 0.9281921387,
        "file_name": "Payment and Online Services Terms Sept 2022.pdf",
        "chunk_id": 116,
        "page": 19,
        "chunk": "dit niet tot korting op het tarief. \n \n4. Wilt u niet langer online bankieren? Dan kan dat tot gevolg hebben dat uw \nbetaalpakket niet meer geschikt is en u een ander pakket moet kiezen. Maakt u \ngeen keuze? Dan mogen wij uw betaalpakket omzetten.\n49. Zonder krediet rood staan op de rekening\n1. Als u geen krediet heeft, kunt u toch rood komen te staan op de rekening. \nBijvoorbeeld doordat wij de kosten van het betaalpakket afboeken van de \nrekening. Wij noemen dat ongeoorloofd rood staan. U betaalt hierover een \nvariabele rente.  \nWij kunnen deze rente altijd wijzigen.\n2. Deze variabele rente is opgebouwd uit de volgende componenten: een \nbasistarief, opslagen in verband met ontwikkelingen op de kapitaalmarkten \nen kapitaalkosten, eventueel individuele risico-opslagen, doorlopende kosten \n(bijvoorbeel

## Gradio Chatbot that uses RAG and GraphRAG

Example code is coming from Gradio documentation: [Creating a custom chatbot with blocks](https://www.gradio.app/guides/creating-a-custom-chatbot-with-blocks#add-streaming-to-your-chatbot)

In [16]:
def user(user_message, history):
    return "", history + [[user_message, None]]

def get_answer(search_prompt, rag_method):
    if rag_method == "Vector-Search":
        context = get_context_vector_search(search_prompt)
        theprompt = generate_prompt_vector_search(search_prompt, context)
    else: 
    # rag_method == "GraphRAG"
        context, definitions = get_context_graphrag(search_prompt)
        theprompt = generate_prompt_graphrag(search_prompt, context, definitions)
    messages = llm(theprompt.to_messages())
    return messages.content

def bot(history, rag_method):
    bot_message = get_answer(history[-1][0], rag_method)
    history[-1][1] = ""
    for character in bot_message:
        history[-1][1] += character
        time.sleep(0.01)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        label="Chatbot with RAG", 
        avatar_images=["https://png.pngtree.com/png-vector/20220525/ourmid/pngtree-concept-of-facial-animal-avatar-chatbot-dog-chat-machine-illustration-vector-png-image_46652864.jpg","https://d-cb.jc-cdn.com/sites/crackberry.com/files/styles/larger/public/article_images/2023/08/openai-logo.jpg"]
    )
    msg = gr.Textbox(label="Message")
    rag_method = gr.Radio(["Vector-Search", "GraphRAG"], label="RAG-method:")
    clear = gr.Button("Clear")


    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot, rag_method], chatbot
    )
    
    clear.click(lambda: None, None, chatbot, queue=False)

    
demo.queue()
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


If you want to have the light-mode for the chatbot paste the following after the URL: /?__theme=light